### Compare_commit_range


In [1]:
from pathlib import Path
from tardisbase.testing.regression_comparison.compare import ReferenceComparer
from tardisbase.testing.regression_comparison.run_tests import run_tests
from tardisbase.testing.regression_comparison.visualization import (
    FileChangeMatrixVisualizer, 
)

In [2]:
config = {
    "tardis_repo_path": "/home/riddhi/workspace/tardis-main/tardis",
    "regression_data_repo_path": "/home/riddhi/workspace/tardis-main/tardis-regression-data",
    "branch": "master",
    "n": 3,
    "target_file": "tardis/spectrum/tests/test_spectrum_solver/test_spectrum_solver/TestSpectrumSolver.h5",
    "commits": ["300e565e83112528faaa76e970057ffb1b13f743", "2a06fdfb60190bbd9b49ff572d78772607138660", "2d775dcd1c486227532f537fc41066e942000e56"],  # Uncomment for specific commits
    # "commits": commits.calculate_commits(n=10, gap=0, info=False), #Uncomment for n commits with custom gap
    "use_conda": True,
    "conda_manager": "conda"
}


In [ ]:
processed_commits, regression_commits, original_head, target_file_path = run_tests(
        config["tardis_repo_path"], 
        config["regression_data_repo_path"], 
        config["branch"], config["target_file"], 
        n=config["n"], 
        use_conda=config["use_conda"],
        conda_manager=config["conda_manager"]
    )

In [3]:
regression_commits = ["b8d2e1c0", "16dcbad6", "70c1e9f4"]

In [3]:
regression_commits = ["4895343020c7280c7dff8a8563e4c0bb19afd55e","d12d869bd2bb2038c9090852ee9ef998959f412d","b008a7180440a697ad5b54a9f77b692d4f71b120","a2a946a43d710c44bb3b08bcae69359fe13ed032","792314720db6505c11ce9f688a4bb8c4827d1a06"]

In [4]:
visualizer_all = FileChangeMatrixVisualizer(
    regression_repo_path=config["regression_data_repo_path"],
    commits=regression_commits
)

# Analyze the commits
visualizer_all.analyze_commits()

# Display the matrix
visualizer_all.print_matrix()

print(f"\nSummary: Analyzed {len(visualizer_all.all_files)} file(s) across {len(regression_commits)} commits")

Analyzing 5 commits...
Processing commit 1/5: 48953430
Processing commit 2/5: d12d869b
Processing commit 3/5: b008a718
Processing commit 4/5: a2a946a4
Processing commit 5/5: 79231472
Found 109 total files across all commits.
Changed files: 28
Unchanged files: 81

Changed Files Matrix (28 files):


,Files,489534,d12d86,b008a7,a2a946,792314
0,atom_data/chianti_He.h5,•,•,•,*,•
1,atom_data/kurucz_cd23_chianti_H_He_latest.h5,∅,∅,∅,+,•
2,tardis/opacities/tests/test_opacity_solver/test_new_macro_atom_solver__downbranch-False__.h5,∅,∅,∅,∅,+
3,tardis/opacities/tests/test_opacity_solver/test_new_macro_atom_solver__downbranch-True__.h5,∅,∅,∅,∅,+
4,tardis/opacities/tests/test_opacity_solver/test_new_macro_atom_solver__macroatom-False__.h5,∅,∅,∅,∅,+
5,tardis/opacities/tests/test_opacity_solver/test_new_macro_atom_solver__macroatom-True__.h5,∅,∅,∅,∅,+
6,tardis/plasma/tests/test_complete_plasmas/test_plasma/plasma_unittest_disable_electron_scattering_False.h5,•,•,•,*,•
7,tardis/plasma/tests/test_complete_plasmas/test_plasma/plasma_unittest_disable_electron_scattering_True.h5,•,•,•,*,•
8,tardis/plasma/tests/test_complete_plasmas/test_plasma/plasma_unittest_excitation_dilute-lte.h5,•,•,•,*,•
9,tardis/plasma/tests/test_complete_plasmas/test_plasma/plasma_unittest_excitation_lte.h5,•,•,•,*,•



Unchanged Files Table (81 files):


,Files,489534,d12d86,b008a7,a2a946,792314
0,atom_data/kurucz_atom_chianti_many.h5,•,•,•,•,•
1,atom_data/kurucz_cd23_chianti_H_He.h5,•,•,•,•,•
2,atom_data/kurucz_cd23_chianti_Si.h5,•,•,•,•,•
3,atom_data/new_kurucz_cd23_chianti_H_He.h5,•,•,•,•,•
4,atom_data/nlte_atom_data/TestNLTE_He_Ti.h5,•,•,•,•,•
5,tardis/opacities/tests/test_tau_sobolev/test_calculate_sobolev_line_opacity.h5,•,•,•,•,•
6,tardis/plasma/equilibrium/tests/test_level_populations/test_level_population_solver/test_solve__collisional_rate_solver0-radiative_transitions0__.h5,•,•,•,•,•
7,tardis/plasma/equilibrium/tests/test_rate_matrix/test_rate_matrix_solver__collisional_rate_solver0-radiative_transitions0__.h5,•,•,•,•,•
8,tardis/plasma/tests/test_complete_plasmas/test_plasma/TestPlasma.h5,•,•,•,•,•
9,tardis/plasma/tests/test_hdf_plasma/test_hdf_levels.h5,•,•,•,•,•



Legend:
──────────────────────────────
  • = unchanged (blue)
  * = modified (gold)
  + = added (green)
  - = deleted (red)
  ∅ = not present (grey)

Summary: Analyzed 109 file(s) across 5 commits
